<a href="https://www.kaggle.com/code/mikedelong/load-clean-and-top-keywords?scriptVersionId=153067275" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
! pip install sentence-transformers 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=a063b37ffcb19e3e244735b00eac93252c17f8a838cfa9ccf4ef91360d5004b2
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [2]:
import pandas as pd
from json import loads
from ast import literal_eval
df = pd.read_csv(filepath_or_buffer='/kaggle/input/text-and-meta-data-analysis/train.csv', )
df['perplexity_score'] = df['meta'].apply(func=lambda x: literal_eval(x)['perplexity_score'])
df['pile_set_name'] = df['meta'].apply(func=lambda x: literal_eval(x)['pile_set_name'])
df = df.drop(columns=['meta'])
df = df.dropna(subset=['text'])
df.head()

,text,perplexity_score,pile_set_name
0,"It is done, and submitted. You can play “Survi...",327.0,Pile-CC
1,"<?xml version=""1.0"" encoding=""UTF-8""?>\r\n<seg...",977.7,Github
2,Article content\n\nHuman behavior has a tremen...,609.4,OpenWebText2
3,Topic: reinvent midnight madness\n\nAmazon ann...,262.3,Pile-CC
4,About Grand Slam Fishing Charters\n\nAs a fami...,858.9,Pile-CC


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 99996 entries, 0 to 99999
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   text              99996 non-null  object 
 1   perplexity_score  99996 non-null  float64
 2   pile_set_name     99996 non-null  object 
dtypes: float64(1), object(2)
memory usage: 3.1+ MB


In [4]:
# Let's clean up the text
df['clean'] = df['text'].str.replace('\n', ' ') # remove embedded newlines
clean_df = df.drop(columns=['text', 'perplexity_score', 'pile_set_name', ]).rename(columns={'clean': 'text'})
clean_df.sample(n=5, random_state=2023)

,text
31934,Flickr Open / Getty Images Old paint peeling f...
18263,Privacy Policy Samparknow is always there to ...
12731,Limited CapacityfullAdding this to your schedu...
76803,"Introduction {#S0001} ============ Recently, ..."
62813,Q: When I run rails server it is showing erro...


In [5]:
# https://www.kaggle.com/code/ianalyticsgeek/keywords-extraction-using-bert
from arrow import now
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

# each of these is somewhat arbitary
N = 20000
MAX_FEATURES = 500
TOP_N = N // 10

# while we are testing let's take a small sample
sample_df = clean_df.sample(n=N, random_state=2023)
fit_data = sample_df['text'].values.tolist()

NGRAM_RANGE = (1, 1)
model_start = now()
count = TfidfVectorizer(ngram_range=NGRAM_RANGE, stop_words='english', max_features=MAX_FEATURES).fit(fit_data)
features = count.get_feature_names_out()
print('{}: got features'.format(now()))
model = SentenceTransformer('distilbert-base-nli-mean-tokens')
embedding = model.encode(fit_data)
print('{}: got embeddings'.format(now()))
feature_embeddings = model.encode(features)
print('{}: got feature embeddings'.format(now()))
print('model time: {}'.format(now() - model_start))

2023-11-30T18:27:06.116869+00:00: got features


Batches:   0%|          | 0/625 [00:00<?, ?it/s]

2023-11-30T19:03:47.464040+00:00: got embeddings


Batches:   0%|          | 0/16 [00:00<?, ?it/s]

2023-11-30T19:03:49.098332+00:00: got feature embeddings
model time: 0:37:06.758384


In [6]:
# post process the features 
# use the cosine distance to measure how far the features are from the documents
distances = cosine_similarity(embedding, feature_embeddings)
# pick out the top few phrases to be prevalent keywords
keep = set([features[index] for index in distances.argsort()[0][-TOP_N:]])
# filter out numeric values
keep = {item for item in keep if not item.isnumeric()}
# do some basic plural identification and remove the plurals 
plurals = {item for item in keep if (item.endswith('s') and item[:-1] in keep) or (item.endswith('es') and item[:-2] in keep)}
keep = {item for item in keep if item not in plurals}
keep = keep.difference(plurals)
print(len(keep), keep)

446 {'period', 'showed', 'void', 'working', 'application', 'thought', 'used', 'considered', 'total', 'does', 'having', 'saw', 'american', 'beta', 'level', 'pi', 'bit', 'decision', 'great', 'close', 'addition', 'object', 'include', 'mathbf', 'section', 'idea', 'tell', 'version', 'energy', 'treatment', 'method', 'small', 'div', 'cost', 'die', 'device', 'compared', 'structure', 'information', 'doesn', 'fig', 'free', 'var', 'defendant', 'group', 'told', 'length', 'review', 'police', 'low', 'community', 'let', 'cause', 'network', 'left', 'hand', 'ref', 'including', 'function', 'age', 'law', 'house', 'going', 'large', 'position', 'pone', 'list', 'shown', 'effect', 'support', 'public', 'issue', 'right', 'material', 'define', 'true', 'described', 'words', 'day', 'operation', 'say', 'date', 'll', 'code', 'shall', 'http', 'different', 'said', 'int', 'man', 'family', 'open', 'set', 'gamma', 'specific', 'use', 'film', 'little', 'eyes', 'seen', 'time', 'log', 'solution', 'simple', 'false', 'private

In [7]:
from nltk import pos_tag
from nltk import word_tokenize

noun_start = now()
keep_nouns = {
    word for item in fit_data for (word, part_of_speech) in pos_tag(word_tokenize(item)) if part_of_speech[:2] == 'NN' and word in keep
}
print('filtered nouns in {}'.format(now() - noun_start))
keep = keep_nouns
print(len(keep), keep)

filtered nouns in 0:29:11.405499
355 {'period', 'void', 'application', 'total', 'saw', 'addition', 'object', 'tell', 'version', 'energy', 'device', 'information', 'doesn', 'defendant', 'police', 'review', 'cause', 'network', 'left', 'hand', 'function', 'law', 'position', 'pone', 'support', 'effect', 'issue', 'material', 'define', 'operation', 'say', 'code', 'll', 'int', 'family', 'set', 'specific', 'eyes', 'time', 'room', 'year', 'action', 'term', 'center', 'range', 'course', 'make', 'taking', 'business', 'path', 'human', 've', 'particular', 'rate', 'best', 'sqrt', 'likely', 'import', 'hard', 'label', 'north', 'way', 'red', 'array', 'shows', 'short', 'force', 'work', 'paper', 'species', 'result', 'check', 'start', 'district', 'protein', 'bar', 'help', 'know', 'id', 'policy', 'number', 'world', 'blood', 'string', 'process', 'delta', 'sigma', 'quite', 'book', 'got', 'static', 'category', 'data', 'death', 'event', 'money', 'field', 'content', 'come', 'error', 'approach', 'half', 'response

In [8]:
# now we can project the feature vectors into 2-space to visualize 
from plotly.express import scatter
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, random_state=2023, verbose=1, n_iter=10000)
tsne_df = pd.DataFrame(data=tsne.fit_transform(X=feature_embeddings), columns=['t0', 't1'])
tsne_df['word'] = features
scatter(data_frame=tsne_df[tsne_df['word'].isin(keep)], x='t0', y='t1', text='word', height=900, ).update_traces(marker={'size': 1})

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 500 samples in 0.001s...
[t-SNE] Computed neighbors for 500 samples in 0.076s...
[t-SNE] Computed conditional probabilities for sample 500 / 500
[t-SNE] Mean sigma: 3.088812
[t-SNE] KL divergence after 250 iterations with early exaggeration: 58.847252
[t-SNE] KL divergence after 2500 iterations: 1.123895


Unlike the word cloud this does a reasonable job at showing what the corpus talks about; it also does a reasonable job of grouping related words together as opposed to laying them out randomly. 